# Pytorch Model
daniel.brooks@alumni.caltech.edu <br>
Feburary 24, 2018 <br>

In [1]:
#Preprocessing imports.
import numpy as np
from sklearn import preprocessing

import draftsimtools as ds

In [2]:
#Torch imports.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
#Load 1000 M19 drafts.
m19_set = ds.create_set("data/m19_rating.tsv", "data/m19_land_rating.tsv")
raw_drafts = ds.load_drafts("data/m19_1000drafts.csv")

In [4]:
#Fix commas in card names.
m19_set, raw_drafts = ds.fix_commas(m19_set, raw_drafts)

In [5]:
#Store rating information in a dictionary.
rating_dict = ds.create_rating_dict(m19_set)

In [6]:
#Process the draft data.
drafts = ds.process_drafts(raw_drafts)

Processing draft: 0.


In [29]:
#Create an M19 player. 
#b = ds.SGDBot(rating_dict)

In [30]:
#%%time
#Optimize rating parameters using stochastic gradient descent. (1 minute / 1000 drafts)
#for x in range(10):
#    b.sgd_optimization(drafts[0:25], 0.05)
#print("Done.")

In [9]:
#Write new ratings to file.
#b.write_rating_dict("sgd_05_linear.tsv")
#b.write_error("error.csv")

# PyTorch Preprocessing

In [10]:
#Save draft data as a tensor.
def create_le(cardnames):
    """Create label encoder for cardnames."""
    le = preprocessing.LabelEncoder()
    le.fit(cardnames)
    return le

def draft_to_matrix(cur_draft, le, pack_size=15):
    """Transform draft from cardname list to one hot encoding."""
    pick_list = [np.append(le.transform(cur_draft[i]), (pack_size-len(x))*[0]) \
                 for i, x in enumerate(cur_draft)]
    pick_matrix = np.int16(pick_list)
    return pick_matrix

def drafts_to_tensor(drafts, le, pack_size=15):
    """Create tensor of shape (num_drafts, 45, 15)."""
    pick_tensor_list = [draft_to_matrix(d, le) for d in drafts]
    pick_tensor = np.int16(pick_tensor_list)
    return pick_tensor

#Create drafts tensor.
le = create_le(m19_set["Name"].values)
drafts_tensor = drafts_to_tensor(drafts, le)

In [32]:
drafts_tensor.shape

(1000, 45, 15)

In [33]:
drafts_tensor[0,:,:]

array([[212,   7,  37, 207, 181, 226, 146, 259,  73, 198, 252,  17, 144,
        114, 126],
       [ 92, 274, 238, 108, 148,  26, 266, 259, 151,  79, 114, 261, 268,
        281,   0],
       [ 89,  69,  97,  79, 216, 253,  33,  62, 268, 190,  23, 155, 165,
          0,   0],
       [ 92, 207, 210, 149, 155, 135, 157, 252,  96,  29, 230,  84,   0,
          0,   0],
       [ 77, 108, 112, 273, 193, 146, 267, 262, 260,  35, 232,   0,   0,
          0,   0],
       [231,  87, 170,  79, 144, 101, 267,   0, 281, 258,   0,   0,   0,
          0,   0],
       [ 92,  31,  75,  39,  43, 114, 118, 252, 187,   0,   0,   0,   0,
          0,   0],
       [262, 273, 208, 253, 131, 281,  96, 163,   0,   0,   0,   0,   0,
          0,   0],
       [ 73, 198, 252,  17, 144, 114, 126,   0,   0,   0,   0,   0,   0,
          0,   0],
       [151,  79, 114, 261, 268, 281,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [268, 190,  23, 155, 165,   0,   0,   0,   0,   0,   0,   0,   0,
       

In [11]:
#Create torch dataset class.
from torch.utils.data.dataset import Dataset

#Drafts dataset class.
class DraftDataset(Dataset):
    """Defines a draft dataset in PyTorch."""
    
    def __init__(self, drafts_tensor, le):
        """Initialization."""
        self.drafts_tensor = drafts_tensor
        self.le = le
        self.cards_in_set = len(self.le.classes_)
        self.pack_size = int(self.drafts_tensor.shape[1]/3)
        
    def __getitem__(self, index):
        """Return a training example.
        """
        #Compute number of picks in a draft.
        draft_size = self.pack_size*3
        
        #Grab information on current draft.
        pick_num = index % draft_size #0-self.pack_size*3-1
        draft_num = int((index - pick_num)/draft_size)
        
        #Generate.
        x = self.create_x(pick_num, draft_num)
        y = self.create_y(pick_num, draft_num)
        return x, y
    
    def create_x(self, pick_num, draft_num):
        """Generate x, input. 
        Column 1: collection vector
                  x[i]=n -> collection has n copies of card i
        Column 2: pack vector
                  0 -> card not in pack
                  1 -> card in pack
        Efficiency optimization possible. Iterative adds to numpy array.
        """
        #Initialize collection / cards in pack vector.
        x = np.zeros([self.cards_in_set, 2], dtype = "int16")
        
        #Fill in collection vector excluding current pick.
        for n in self.drafts_tensor[draft_num, :pick_num, 0]:
            x[n, 0] += 1
            
        #Fill in pack vector.
        cards_in_pack =  self.pack_size - pick_num%self.pack_size #Cards in current pack.
        for n in self.drafts_tensor[draft_num, pick_num, :cards_in_pack]:
            x[n, 1] = 1
        
        #Convert to Torch tensor. 
        #print("TEST: LEAVING OUT PACK VECTOR.")
        #x = torch.Tensor(x[:,0])
        x = torch.Tensor(x)
        return x
    
    def create_y(self, pick_num, draft_num, not_in_pack=0.5):
        """Generate y, a target pick vector.
        Picked card is assigned a value of 1.
        Other cards are assigned a value of 0.
        """
        #Initialize target vector.
        #y = np.array([0] * self.cards_in_set)
        y = np.zeros([self.cards_in_set, 1], dtype = "int16")
            
        #Add picked card.
        y[self.drafts_tensor[draft_num, pick_num, 0]] = 1
        y = torch.Tensor(y)
        return y
    
    def __len__(self):
        return len(self.drafts_tensor)
    
#Create a draft dataset.
d = DraftDataset(drafts_tensor, le)

In [12]:
#Explore drafts dataset.
#x,y = d.__getitem__(45)
#print("x =", x, "\n")
#print("y =", y)

In [13]:
#Implement NN.
class Net(nn.Module):
    
    def __init__(self, ss):
        """param ss: number of cards in set
        """
        super(Net, self).__init__()
        self.fc1 = nn.Linear(ss,ss)
        self.fc2 = nn.Linear(ss,ss)
        self.sm = nn.Softmax(dim=0)
        
    def forward(self, x):
        #x0 = x[:, 0] #Collection
        #x1 = x[:, 1] #Cards in pack
        
        #Desired forward prop function.
        out = F.relu(self.fc1(x[:,0]))
        out = F.relu(self.fc2(out))
        out = out * x[:,1] #Constrain cards based on what is in pack.
        out = self.sm(out)
        #out = x
        
        return out

#Create NN.
net = Net(len(m19_set))
print(net)

Net(
  (fc1): Linear(in_features=285, out_features=285, bias=True)
  (fc2): Linear(in_features=285, out_features=285, bias=True)
  (sm): Softmax()
)


In [14]:
###############################
# Define training parameters. #
###############################

#Define training set. Batchsize must be 1.
trainset = d
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True)

#Loss function.
criterion = nn.MSELoss()
#criterion = nn.CrossEntropyLoss()

#Define optimizer. 
optimizer = optim.SGD(net.parameters(), lr=0.5)
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [23]:
d.create_x()

In [15]:
# Train network.

#Set up pulled from pytorch tutorial.
#https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-download-beginner-blitz-cifar10-tutorial-py

NUM_EPOCH = 10
num_draft = len(trainloader)

for epoch in range(NUM_EPOCH):
    
    #Loop over x,y for each dataset.
    running_loss = 0
    for i, data in enumerate(trainloader, 0):
        
        #Get the inputs.
        #inputs, labels = data
        inputs = data[0][0,:,:]
        labels = data[1][0,:,:]
                
        #Zero parameter gradients between batches.
        optimizer.zero_grad()
        
        #Perform training.
        outputs = net(inputs) #Required batch size of 1.
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #Print loss data.
        running_loss += loss.item()
        if i % num_draft == num_draft-1:
            print('Epoch %d, Average Loss: %.6f' % (epoch+1, running_loss/num_draft))
            running_loss = 0.0
            
print("Finished Training")

Epoch 1, Average Loss: 0.003496
Epoch 2, Average Loss: 0.003496
Epoch 3, Average Loss: 0.003496
Epoch 4, Average Loss: 0.003496
Epoch 5, Average Loss: 0.003496
Epoch 6, Average Loss: 0.003496
Epoch 7, Average Loss: 0.003496
Epoch 8, Average Loss: 0.003496
Epoch 9, Average Loss: 0.003496
Epoch 10, Average Loss: 0.003496
Finished Training


In [16]:
inputs.shape

torch.Size([285, 2])

In [17]:
labels.shape

torch.Size([285, 1])

In [18]:
#Now evaluate network over all possible picks. 
for i, data in enumerate(trainloader, 0):
    if i<1:
        zz = net(data[0][0,:,:])
        print(zz)

tensor([0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035,
        0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035,
        0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035,
        0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035,
        0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035,
        0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035,
        0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035,
        0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035,
        0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035,
        0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035,
        0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035,
        0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0037, 0.0035,
        0.0035, 0.0035, 0.0035, 0.0035, 

In [19]:
max(zz)

tensor(0.0037, grad_fn=<SelectBackward>)

In [20]:
min(zz)

tensor(0.0035, grad_fn=<SelectBackward>)